In [ ]:
pip install chromadb # Python에 ChromaDB 설치

In [ ]:
import chromadb
client = chromadb.PersistentClient() # 디스크 모드로 이용, 이외에도 인메모리, HttpClient 방식 등이 존재

In [ ]:
# 컬렉션 생성
answers = client.create_collection(
    name="answers"
)

In [ ]:
import pandas as pd
from tqdm import tqdm

"""
    게시글 CNU DATA 불러오기
"""

df = pd.read_csv("./cnu_post_data.csv")
df.sample(5)

In [ ]:
from langchain.embeddings import LlamaCppEmbeddings

llama_model_path = "umm.."
embeddings = LlamaCppEmbeddings(model_path=llama_model_path,n_ctx=2048)

In [ ]:
from sentence_transformers import SentenceTransformer
"""
    벡터 임베딩을 위한 SentenceTransformer model 불러오기
    Langchain이 지원하는 벡터 임베딩 방법이 있나?
"""

model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS') 

In [ ]:
ids = []
metadatas = []
embeddings = []

for row in tqdm(df.iterrows()):
    index = row[0]
    query = row[1].user # xlsx 예시
    answer = row[1].answer # xlsx 예시
    
    metadata = {
        "query": query,
        "answer": answer
    }
    
    embedding = model.encode(query, normalize_embeddings=True)
    
    ids.append(str(index))
    metadatas.append(metadata)
    embeddings.append(embedding)
    
chunk_size = 1024  # 한 번에 처리할 chunk 크기 설정
total_chunks = len(embeddings) // chunk_size + 1  # 전체 데이터를 chunk 단위로 나눈 횟수
embeddings = [ e.tolist() for e in tqdm(embeddings)]  

for chunk_idx in tqdm(range(total_chunks)):
    start_idx = chunk_idx * chunk_size
    end_idx = (chunk_idx + 1) * chunk_size
    
    # chunk 단위로 데이터 자르기
    chunk_embeddings = embeddings[start_idx:end_idx]
    chunk_ids = ids[start_idx:end_idx]
    chunk_metadatas = metadatas[start_idx:end_idx]
    
    # chunk를 answers에 추가
    answers.add(embeddings=chunk_embeddings, ids=chunk_ids, metadatas=chunk_metadatas)

In [ ]:

# 검색하기
# answers = create_collection으로 생성한 chroma DB의 객체
result = answers.query(
    query_embeddings=model.encode("어제 여자친구랑 헤어졌다", normalize_embeddings=True).tolist(),
    n_results=3
)


print(result)